## SURF (Speeded-Up Robust Features)

### Import resources and display image

In [ ]:
!pip3 install opencv-python

In [ ]:
!pip install scikit-image

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib
from skimage import io
import time

%matplotlib inline

In [ ]:
# self-written scripts
import sys
sys.path.insert(0, 'Python_Scripts')

import util
import surf_handling

In [ ]:
# Load the image
image1 = cv2.imread('data/train_images/0002cc93b.jpg')

# Convert the training image to RGB
training_image = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)

# Convert the training image to gray scale
training_gray = cv2.cvtColor(training_image, cv2.COLOR_RGB2GRAY)

# Create test image by adding Scale Invariance and Rotational Invariance
test_image = cv2.pyrDown(training_image)
test_image = cv2.pyrDown(test_image)
num_rows, num_cols = test_image.shape[:2]

rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), 30, 1)
test_image = cv2.warpAffine(test_image, rotation_matrix, (num_cols, num_rows))

test_gray = cv2.cvtColor(test_image, cv2.COLOR_RGB2GRAY)

# Display traning image and testing image
fx, plots = plt.subplots(2, 1, figsize=(20,10))

plots[0].set_title("Training Image")
plots[0].imshow(training_image)

plots[1].set_title("Testing Image")
plots[1].imshow(test_image)

### Detect keypoints and Create Descriptor

In [ ]:
surf = cv2.xfeatures2d.SURF_create(400)

train_keypoints, train_descriptor = surf.detectAndCompute(training_gray, None)
test_keypoints, test_descriptor = surf.detectAndCompute(test_gray, None)

keypoints_without_size = np.copy(training_image)
keypoints_with_size = np.copy(training_image)

cv2.drawKeypoints(training_image, train_keypoints, keypoints_without_size, color = (0, 255, 0))

cv2.drawKeypoints(training_image, train_keypoints, keypoints_with_size, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# Display image with and without keypoints size
fx, plots = plt.subplots(2, 1, figsize=(20,10))

plots[0].set_title("Train keypoints With Size")
plots[0].imshow(keypoints_with_size, cmap='gray')

plots[1].set_title("Train keypoints Without Size")
plots[1].imshow(keypoints_without_size, cmap='gray')

# Print the number of keypoints detected in the training image
print("Number of Keypoints Detected In The Training Image: ", len(train_keypoints))

# Print the number of keypoints detected in the query image
print("Number of Keypoints Detected In The Query Image: ", len(test_keypoints))

### Matching Keypoints

In [ ]:
# Create a Brute Force Matcher object.
bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck = False)

# Perform the matching between the SURF descriptors of the training image and the test image
matches = bf.match(train_descriptor, test_descriptor)

# The matches with shorter distance are the ones we want.
matches = sorted(matches, key = lambda x : x.distance)

result = cv2.drawMatches(training_image, train_keypoints, test_gray, test_keypoints, matches, test_gray, flags = 2)

# Display the best matching points
plt.rcParams['figure.figsize'] = [14.0, 7.0]
plt.title('Best Matching Points')
plt.imshow(result)
plt.show()

# Print total number of matching points between the training and query images
print("\nNumber of Matching Keypoints Between The Training and Query Images: ", len(matches))

In [ ]:
training_gray

In [ ]:
img = cv2.imread('data/train_images/0002cc93b.jpg',0)
# Create SURF object. You can specify params here or later.
# Here I set Hessian Threshold to 400
surf = cv2.xfeatures2d.SURF_create(400)
# Find keypoints and descriptors directly
kp, des = surf.detectAndCompute(img,None)
len(kp)

In [ ]:
# Check present Hessian threshold
print( surf.getHessianThreshold() )

In [ ]:
# We set it to some 50000. Remember, it is just for representing in picture.
# In actual cases, it is better to have a value 300-500
surf.setHessianThreshold(300)
# Again compute keypoints and check its number.
kp, des = surf.detectAndCompute(img,None)
print( len(kp) )


In [ ]:
img2 = cv2.drawKeypoints(img,kp,None,(255,0,0),4)
plt.imshow(img2),plt.show()

In [ ]:
# Find size of descriptor
print( surf.descriptorSize() )

In [ ]:
# That means flag, "extended" is False.
surf.getExtended()

In [ ]:
# So we make it to True to get 128-dim descriptors.
surf.setExtended(True)
kp, des = surf.detectAndCompute(img,None)
print( surf.descriptorSize() )

In [ ]:
print( des.shape )

In [ ]:
img3 = cv2.drawKeypoints(img,kp,None,(255,0,0),4)
plt.imshow(img3),plt.show()

---

### SURF-Attributes

In [ ]:
print('Keypoint at:', kp[0].pt)
print('Keypoint diameter:', kp[0].size)
print('Direction of gradient:', kp[0].angle)

In [ ]:
len(kp)

---

### Apply SURF to all images

In [ ]:
"""Processing time: ~3 minutes and 40 seconds.
"""

# get current working directory
cwd = pathlib.Path.cwd()
train_data_dir = cwd.joinpath('data', 'train_images')
train_images = list(train_data_dir.glob('*.jpg'))

# Create SURF object. You can specify params here or later.
# Here I set Hessian Threshold to 400
surf = cv2.xfeatures2d.SURF_create(400)

# prepare dictionary to gather data
surf_images = {'keypoints': [],
               'ImageId': [],
               'NumberKP': []
              }

print('processing images...')
start = time.time()

for idx, image in enumerate(train_images):
    surf_images['ImageId'].append(image.name)
    
    # `image` so far holds just the path to the image. Convert to image file
    image = io.imread("data/train_images/"+image.name)
    # Find keypoints and descriptors directly
    kp, des = surf.detectAndCompute(image, None)
    
    surf_images['keypoints'].append(kp)
    surf_images['NumberKP'].append(len(kp))
    if idx % 500 == 0 and idx != 0:
        print(f'image number {idx} processed...')

end = time.time()
print('processing done.')
print('required time:', end - start)

In [ ]:
temp = pd.DataFrame.from_dict(surf_images)
temp.head()

In [ ]:
print(f"We have {temp.query('NumberKP < 50').count()[0]} keypoint vectors with less than 50 keypoints")

In [ ]:
temp.sort_values(by='NumberKP', ascending=False)

Adjust data frame and eliminate images that have more than 1 defect.

In [ ]:
util.isolate_single_defects(temp)

---

### Prepare data frame with (max) TOP50 Keypoints per image

In [ ]:
"""Expected run-time: ~ 35 minutes
"""
# apply functions to data frame
temp = surf_handling.add_keypoint_parameters(temp)

---

### Save data frame for further processing

In [ ]:
temp.to_csv('data/train_surf.csv', sep=',', index=False)

---

### Trials

In [ ]:
x = pd.DataFrame(columns = ['x'])
y = pd.DataFrame(columns = ['y'])
s = pd.DataFrame(columns = ['s'])

total_time = 0

start = time.time()
x.x = temp.keypoints.apply(lambda x: get_keypoint_x(x, 1))
end = time.time()
print('processing time x:', end-start)
total_time += end - start

start = time.time()
y.y = temp.keypoints.apply(lambda x: get_keypoint_y(x, 1))
end = time.time()
print('processing time y:', end-start)
total_time += end - start

start = time.time()
s.s = temp.keypoints.apply(lambda x: get_keypoint_size(x, 1))
end = time.time()
print('processing time s:', end-start)
total_time += end - start

start = time.time()
temp = pd.concat([temp, x, y, s], axis=1, ignore_index=False)

end = time.time()
print('processing time concat:', end-start)
total_time += end - start
print('total processing time was:', total_time)
